# MCTS TO-DO
Implement a Monte-Carlo Tree Search Agent that selects actions based on reward to replicate Three Sisters.

 *Goals*
- edit environment so it is not random i.e. agent choose to location of plants
- define policy to select state
- define how to evalulate a simulation
- return best simulation and fine tune

*Sunday 2/27*
- Define basic agent that uses Adam's value function
- Get agent to return best simulation

*Monday 2/28* 
Updates
- made agent class
- traversal, rollout, backprop code added
- state class defined 

TO-DO
- Add tree to agent
- Edit environment to reflect precised planting
- Finish randomized action based on new environment


In [1]:
import numpy as np
import math

class MCTS_Agent:

    ##root is a Node object that defines the start of the tree to be traversed
    def __init__(self, root, field, it, limit=10):
        self.root = root
        self.test_bed = field
        self.it = it
        self.sow_limit = limit

    ##Run the mcts traversal for the specified number of iterations
    def mcts_traversal(self):
        for i in range(self.it):
            self.test_bed.reset()
            node = self.root
            path = [] # need to use path

            while(node.explored() and not(node.is_terminal())):
                best_child = node
                best_score = 0
                for child in node.children:
                    score = child.get_ucb()
                    if (score > best_score):
                        best_score = score
                        best_child = child
                path.append(best_child)
                best_child.visit()

                action = node.get_action()
                self.test_bed.step(action)
                node = best_child

            if (not(node.is_terminal())):
                move = node.rand_action() ##What is the maximum dimensions we want to use? Currently setting at 10 x 10
                self.test_bed.step(move)
                new_node = state(self.test_bed, move, node, False, False)
                node.add_child(new_node)
                path.append(new_node)
                outcome = self.random_sow(new_node)

            else:
                outcome = node.calc_avg() ## TODO: Check, calc_avg() or get_reward() ??
            self.backprop(path, outcome)

    def random_sow(self, start): # runs monte carlo simulation by picking random moves from root to terminal, will call backprop()
        curr =  start
        done = False
        reward = 0
        while not(done):
            #randomly select an action and make a new state
            action = curr.rand_action()
            observation, reward, done, _ = self.test_bed.step(action)
            new_node = state(observation, action, curr, False, False)
            curr = new_node
        curr.set_term(True)
        return reward


    def backprop(self, path, reward): # updates heuristic (UCB) and returns the updated path

        for node in path:
            node.update(reward)

        #if we do node based traversal
        # while node.parent:
            #node.visit()
            #node.calcUCB()
            #node = node.parent
        return path

    ## Should return the best path discovered so far
    def get_best_path(self):
        path = []
        node = self.root
        while not(len(node.children) == 0):
            best_child = node
            best_score = 0
            for child in node.children:
                score = child.get_ucb()
                if (score > best_score):
                    best_score = score
                    best_child = child
            path.append(best_child)
            node = best_child
        return path


In [ ]:
class state:
    def __init__(self, field, action, pred, term, root):
        self.field = field  
        self.action = action
        self.parent = pred
        self.children = []
        self.terminal =  term
        self.is_root = root
        
        # calculating heuristic
        self.val = 0 # value of a node i (total yield)
        self.avg = 0 # empirical mean of a node i 
        self.c = 0.1 # constant for UCB: range is 0-1, experiment w different values
        self.t = 1 # total number of simulations
        # may need to add more data members 

    def set_term(self, bool):
        self.isTerminal = bool

    def add_child(self,state):
        self.children.append(state)

    def get_ucb(self):
        if self.is_root:
            return self.val/self.t
        return self.val/self.t + self.c* math.sqrt(math.log(self.parent.t)/self.t)

    def get_reward(self):
        return self.val

    def get_action(self):
        return self.action

    def visit(self):
        self.t +=1
    
    def update(self, reward):
        self.val += reward

    def is_terminal(self):
        return self.terminal

    def calc_avg(self): # empirical mean of state
        return self.val/self.t

    def rand_action(self):
        action = np.ones((10,3))
        for i in range(10):
            action[i][0] = np.random.randint(4)
            action[i][1] = np.random.randint(10)
            action[i][2] = np.random.randint(10)
        return action
    
    def explored(self):
        return len(self.children) == 400


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym import error, spaces, utils
from gym.utils import seeding
from enum import Enum

class Plant:
    def __init__(self, species, maturity=110):
        self.species = species
        self.maturity = maturity         # consider 'days_to_maturity'
        self.age = 0
        
    def __repr__(self):
        return "{}".format(self.species)
    

class Field(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, size=5, sow_limit=200, season=120, calendar=0):
        # parameters for overall field character
        self.size = size
        self.sow_limit = sow_limit
        self.season = season
        self.calendar = calendar
        
        # constants for computing end-of-season reward---distances represent meters
        self.crowding_dist = .02
        self.maize_maize_dist = .1
        self.bean_support_dist = .1
        self.crowding_penalty = .1
        self.maize_maize_penalty = .9
        self.bean_support_bonus = .6
        
        # OpenAI action and observation space specifications
        self.action_space = spaces.Discrete(4)
        ### self.observation_space = spaces.???
        
        # field is initialized by calling reset()
        self.field = None
        
    def step(self, action):
        # sow plants (or wait) depending on actions chosen
        # action is an array of n choices; value of n specified in agent code sow_limit
        # could be cleaned up with plants as an enumeration?
        
                   
             ## this part of the code is a work in progress!!   
             ##------------------START OF WIP------------------------------------   
                    
             #declare a new variable that will be the result of the computer figuring out where we want to plant   
             #developed_coord = curr   
             #curr is the result from rollout   
           
             #planttypeTuple = ("Maize", "Bean", "Squash")   
                
             #coordTuple = curr.strip().split()    
                
             #self.field = np.append(self.field, [[self.size*(coordTuple[0]), self.size*(coordTuple[1]), self.size*(coordTuple[2]), Plant(planttypeTuple[0]), Plant(planttypeTuple[1], Plant(planttypeTuple[2]))])   ,
                
            
            ###Experiment: each choice should be represented as an array with 3 elements:
            ### plant choice, y coordinate, x coordinate (in that order).
            ### i.e. action should look like: [[choice1, y1, x1], [choice2, y2, x2]...]
               
        for choice in action:
            if choice[0] == 0:
                self.field = np.append(self.field, [[self.size * choice[1], 
                                                 self.size * choice[2], 
                                                 Plant('Maize')]], axis=0)
            elif choice[0] == 1:
                self.field = np.append(self.field, [[self.size * choice[1], 
                                                 self.size * choice[2], 
                                                 Plant('Bean')]], axis=0)
            elif choice[0] == 2:
                self.field = np.append(self.field, [[self.size * choice[1], 
                                                 self.size * choice[2], 
                                                 Plant('Squash')]], axis=0)
            # when choice == 3, nothing is done (agent waits)   

             ##--------------------------END OF WIP----------------------------------   
                                                    
     #         for choice in action:   
     #             if choice == 0:   
     #                 self.field = np.append(self.field, [[self.size * coordTuple,    
     #                                              self.size * coordTuple,    
     #                                              Plant('Maize')]], axis=0)   
        
     #             elif choice == 1:   
     #                 self.field = np.append(self.field, [[self.size * input(),    
     #                                              self.size * input(),    
     #                                              Plant('Bean')]], axis=0)   
     #             elif choice == 2:   ,
     #                 self.field = np.append(self.field, [[self.size * input(),    
     #                                              self.size * input(),    
     #                                              Plant('Squash')]], axis=0)   
            
        
        # increment timekeeping
        self.calendar +=1
        for plant in self.field:
            plant[2].age += 1
            
        done = self.calendar == self.season
            
        if not done:
            reward = 0
        else:
            reward = self.get_reward()
            
        return self.field, reward, done, {}
    
    def reset(self):
        # field is initialized with one random corn plant in order to make sowing (by np.append) work
        self.field = np.array([[self.size * np.random.random(), 
                                self.size * np.random.random(), 
                                Plant('Maize')]])
        # timekeeping is reset
        self.calendar = 0
        
    def render(self, mode='human'):
        # initialize plant type arrays so that pyplot won't break if any is empty
        maize = np.array([[None, None]])
        bean = np.array([[None, None]])
        squash = np.array([[None, None]])
        maize_imm = np.array([[None, None]])
        bean_imm = np.array([[None, None]])
        squash_imm = np.array([[None, None]])
        
        # replace initial arrays with coordinates for each plant type; imm are plants that haven't matured
        maize = np.array([row for row in self.field 
                             if row[2].__repr__() == 'Maize' and row[2].age >= row[2].maturity])
        bean = np.array([row for row in self.field 
                            if row[2].__repr__() == 'Bean' and row[2].age >= row[2].maturity])
        squash = np.array([row for row in self.field 
                              if row[2].__repr__() == 'Squash' and row[2].age >= row[2].maturity])
        maize_imm = np.array([row for row in self.field 
                             if row[2].__repr__() == 'Maize' and row[2].age < row[2].maturity])
        bean_imm = np.array([row for row in self.field 
                             if row[2].__repr__() == 'Bean' and row[2].age < row[2].maturity])
        squash_imm = np.array([row for row in self.field 
                             if row[2].__repr__() == 'Squash' and row[2].age < row[2].maturity])
        
        # plot the field---currently breaks if any plant type is absent
        plt.figure(figsize=(10, 10))
        plt.scatter(maize[:,0], maize[:,1], c='green', s=200, marker = 'o', alpha=.5, edgecolor='#303030')
        plt.scatter(bean[:,0], bean[:,1], c='brown', s=150, marker = 'o', alpha=.5, edgecolor='#303030')
        plt.scatter(squash[:,0], squash[:,1], c='orange', s=400, marker = 'o', alpha=.5, edgecolor='#303030')
        plt.scatter(maize_imm[:,0], maize_imm[:,1], c='green', s=200, marker = 'o', alpha=.1, edgecolor='#303030')
        plt.scatter(bean_imm[:,0], bean_imm[:,1], c='brown', s=200, marker = 'o', alpha=.1, edgecolor='#303030')
        plt.scatter(squash_imm[:,0], squash_imm[:,1], c='orange', s=200, marker = 'o', alpha=.1, edgecolor='#303030')

        plt.show()
        
        print("Total yield in Calories is {}.\n---\n".format(round(reward, 1)))
    
    def close(self):
        # unneeded right now? AFAICT this is only used to shut down realtime movie visualizations
        pass
    
    def get_reward(self):
        # array of plant coordinates for computing distances
        xy_array = np.array([[row[0], row[1]] for row in self.field])

        # distances[m,n] is distance from mth to nth plant in field
        distances = np.linalg.norm(xy_array - xy_array[:,None], axis=-1)
        
        reward = 0
        i = 0
        while i < len(self.field):
            if self.field[i,2].age < self.field[i,2].maturity:
                reward += 0
            elif self.field[i,2].__repr__() == 'Maize':
                cal = 1
                j = 0
                while j < len(distances[0]):
                    if (self.field[j,2].__repr__() == 'Bean' 
                            and distances[i,j] < self.bean_support_dist):
                        cal += self.bean_support_bonus
                    if (self.field[j,2].__repr__() == 'Maize' 
                            and i !=j 
                            and distances[i,j] < self.maize_maize_dist):
                        cal *= self.maize_maize_penalty
                    if 0 < distances[i,j] < self.crowding_dist:
                        cal *= self.crowding_penalty
                    j += 1
                reward += cal
            elif self.field[i,2].__repr__() == 'Bean':
                reward += .25
            elif self.field[i,2].__repr__() == 'Squash':
                reward += 3
            i += 1        
        return reward


In [17]:
field = Field()
field.reset()
test = MCTS_Agent(state(field, [], None, False, True), field, 1)
test.mcts_traversal()

actions = test.get_best_path()
print(actions)

In [ ]:
field = Field()
field.reset()

test_agent = MCTS_Agent(state(field, [], None, False, True), field, 10000)
test_agent.mcts_traversal()


print(test_agent)



TypeError: __init__() missing 1 required positional argument: 'root'